# test: test_jsonpath

[test_jsonpath](../api/tasks.rst#nornir_tests.plugins.tests.test_jsonpath) uses jsonpath in conjunction with assertpy to validate task data.

This module combines the power of jsonpath and assertpy.  In order to use this module it is somewhat necessary to be familiar with both of those.  Links to them can be found below.

* `jsonpath_ng <https://github.com/h2non/jsonpath-ng>`
* `assertpy <https://github.com/assertpy/assertpy>`


Examples:

In [251]:
from nornir import InitNornir
from nornir_napalm.plugins.tasks import napalm_get, napalm_cli

from nornir_tests.plugins.functions import print_result
from nornir_tests.plugins.tasks import wrap_task
from nornir_tests.plugins.tests import test_jsonpath

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "inventory/hosts.yaml",
            "group_file": "inventory/groups.yaml",
            "defaults_file": "inventory/defaults.yaml",
        }
    },
    dry_run=True,
)

When running a task, validations usually if done in Nornir can be executed as additional logic implemented in python or with running of actual tasks.  Using `nornir_tests` moves the logic into the task and provides a way to impact the success of a task based on its validations.

In [252]:
results = nr.run(
    wrap_task(napalm_get), getters=['facts'],
    tests=[
        test_jsonpath(path='$..os_version', value='4.14.3-2329074.gaatlantarel')
    ]
)

In [253]:
print_result(results, vars=['tests'])

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'is_equal_to',
 'path': '$..os_version',
 'result_attr': 'result',
 'value': '4.14.3-2329074.gaatlantarel'}
{'matches': ['facts.os_version']}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'is_equal_to',
 'path': '$..os_version',
 'result_attr': 'result',
 'value': '4.14.3-2329074.gaatlantarel'}
{'matches': ['facts.os_version']}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


The first example was pretty simple but the next will have many tests run in validating interface data.  It will also use @ decorator syntax.

In [254]:
@test_jsonpath(path='$..ipv6', assertion='contains', value="1::1")
@test_jsonpath(path='$.interfaces_ip', assertion='is_length', value=3)
@test_jsonpath(path='$..FastEthernet8..prefix_length', value=22)
def get_interface_ips(task):
    return napalm_get(task, getters=['interfaces_ip'])

results = nr.run(get_interface_ips)

In [255]:
print_result(results, vars=['tests'])

get_interface_ips***************************************************************
* rtr00 ** changed : False *****************************************************
vvvv get_interface_ips ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'is_equal_to',
 'path': '$..FastEthernet8..prefix_length',
 'result_attr': 'result',
 'value': 22}
{'matches': ['interfaces_ip.FastEthernet8.ipv4.10.66.43.169.prefix_length']}
P JsonPathRecord - {'assertion': 'is_length',
 'path': '$.interfaces_ip',
 'result_attr': 'result',
 'value': 3}
{'matches': ['interfaces_ip']}
P JsonPathRecord - {'assertion': 'contains',
 'path': '$..ipv6',
 'result_attr': 'result',
 'value': '1::1'}
{'matches': ['interfaces_ip.Loopback555.ipv6']}
^^^^ END get_interface_ips ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv get_interface_ips ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

The next example will show how a task can be set to failed based on the validation that is performed.  If the task has fail_task set to true and ends up with passed=False in the test it will mark the overall task as failed.

In [256]:
@test_jsonpath(path='$..connection_state', value="Established", fail_task=True)
def check_bgp_neighbors(task):
    return napalm_get(task, getters=['bgp_neighbors_detail'])

results = nr.run(check_bgp_neighbors)
print_result(results, vars=['tests'])

check_bgp_neighbors*************************************************************
* rtr00 ** changed : False *****************************************************
vvvv check_bgp_neighbors ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'is_equal_to',
 'fail_task': True,
 'path': '$..connection_state',
 'result_attr': 'result',
 'value': 'Established'}
{'matches': ['bgp_neighbors_detail.global.8121.[0].connection_state']}
^^^^ END check_bgp_neighbors ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv check_bgp_neighbors ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
F JsonPathRecord - {'assertion': 'is_equal_to',
 'fail_task': True,
 'path': '$..connection_state',
 'result_attr': 'result',
 'value': 'Established'}
{'exception': Exception(AssertionError('Expected <EstabSync> to be equal to <Established>, but was not.'))}
^^^^ END check_bgp_nei

The last example will use host_data in order to find some data specific to the host to validate against.  The host_data is anything from inventory perhaps obviously in the data dictionary.

In [257]:
nr.data.reset_failed_hosts()

results = nr.run(
    wrap_task(napalm_get), getters=['interfaces'],
    tests=[
        test_jsonpath(path='$.interfaces', assertion='contains', host_data='$.mgmt_port')
    ]
)
print_result(results, vars=['tests'])

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'contains',
 'host_data': '$.mgmt_port',
 'path': '$.interfaces',
 'result_attr': 'result',
 'value': 'Management1'}
{'matches': ['interfaces']}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'contains',
 'host_data': '$.mgmt_port',
 'path': '$.interfaces',
 'result_attr': 'result',
 'value': 'Management2'}
{'matches': ['interfaces']}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In these examples 'contains', 'is_equal', and 'is_length' were used for assertions.  Many other possibilities are available from the assertpy module.  Not saying all of them make sense to use or that they all work as expected but they should.  Too many to validate to be honest.  Some other that would certainly work fine would be 'is_true', 'is_empty', etc.

A few more things about how it all works.  The one_of argument isn't always needed but it could be.  If the match was intended to turn up many of something and something like is_equal assertion is used, if one_of is not true then it will fail if all matches don't meet the assertion.  This is kind of confusing and I should prob show an example here.

In [258]:
@test_jsonpath(path='$..ipv4', assertion='contains', value="10.66.43.169")
@test_jsonpath(path='$..ipv4', assertion='contains', value="10.66.43.169", one_of=True)
def get_interface_ips(task):
    return napalm_get(task, getters=['interfaces_ip'])

rtr00 = nr.filter(name='rtr00')
results = rtr00.run(get_interface_ips)
print_result(results, vars=['tests'])

get_interface_ips***************************************************************
* rtr00 ** changed : False *****************************************************
vvvv get_interface_ips ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JsonPathRecord - {'assertion': 'contains',
 'one_of': True,
 'path': '$..ipv4',
 'result_attr': 'result',
 'value': '10.66.43.169'}
{'matches': ['interfaces_ip.FastEthernet8.ipv4']}
F JsonPathRecord - {'assertion': 'contains',
 'path': '$..ipv4',
 'result_attr': 'result',
 'value': '10.66.43.169'}
{'exception': Exception(AssertionError("Expected <{'192.168.1.1': {'prefix_length': 24}}> to contain key <10.66.43.169>, but did not.")),
 'matches': ['interfaces_ip.FastEthernet8.ipv4']}
^^^^ END get_interface_ips ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


So what happened is the first test passed as it found a bunch of paths that ended with 'ipv4' and it only needed one of them to contain the value of "10.66.43.169".  The second one failed due to the fact that it wanted all the paths to contain that value and they did not.